## Машинный перевод на Seq2Seq с использованием torchtext

Один из самых известных параллельных датасетов для машинного перевода - [Tatoeba](https://tatoeba.org/en/). В пригодном для обучения виде его можно найти [тут](https://www.manythings.org/anki/). Я для удобства сразу разделила файл на трейн и тест (можно это самостоятельно сделать в колабе):

In [ ]:
!wget -qq http://www.manythings.org/anki/rus-eng.zip 
!unzip rus-eng.zip

with open('rus.txt') as f:
    lines = f.readlines() 

eighty = int(len(lines) * 0.8)

with open('train.txt', 'w') as f:
    for line in lines[:eighty]:
        f.write(line)

with open('test.txt', 'w') as f:
    for line in lines[eighty:]:
        f.write(line)

Для того, чтобы удобным образом делать датасеты и даталоудеры в торче из текстов, существует библиотека torchtext. В ее ранних версиях (примерно до 0.6) вы можете видеть такие классы, как Field, Example, Dataset, BucketIterator - они все были выпилены из более поздних версий, так что если видите какой-нибудь туториал с этими классами, устанавливайте torchtext==0.10.0 или старше. 

В современных версиях разрабы торча, как обычно, полностью поломали всю обратную совместимость, какая и была, теперь у нас совсем поменялись принципы работы с текстовыми данными. 

Для того, чтобы обучить сеточку переводить, нам нужно будет сделать такой датасет, который будет плеваться парами предложений source-target, при этом они должны быть токенизированы и добиты паддингом. И не забудем, что нам нужен словарь: для того, чтобы эти самые токены превратить в индексы. 

Для создания словаря и для загрузки такого датасета с парами предложений у torchtext есть свои инструменты. Заимпортим их. Нам также понадобится spacy для токенизации, для него еще нужно загрузить соответствующие модельки:

In [104]:
from torchtext.data.utils import get_tokenizer          # будет брать токенизатор из спейси и пихать его в collate_fn
import torchdata.datapipes as dp                        # будет делать итератор из нашего текстового файла
from torchtext.vocab import build_vocab_from_iterator   # будет делать словарь
from torch.nn.utils.rnn import pad_sequence             # будет падить
from torch.utils.data import DataLoader                 # будет грузить батчи
import torch.nn.functional as F
import spacy 

# заодно импортнем все нужное для модельки
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

# это просто время засекать
from timeit import default_timer as timer

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm

Потом нам надо будет задать некоторые важные вещи, которые мы будем использовать, и подготовить себе инструменты для токенизации и собственно словари (их по два - ведь у нас два языка. )

In [2]:
# Зададим языки
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'ru'

token_transform = {}
vocab_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='ru_core_news_sm')

# Зададим спецсимволы, чтобы потом их раскодировывать
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Не перепутайте порядок - они в таком порядке и добавятся в словарь
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

f:\CODE\Python\Interpreters\Python310\lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


Теперь подготовим итератор для датасета и соберем словарь. 

In [55]:
def getTokens(data_iter, place):
    """
    Возвращаем списки токенов для соответствующих языков: 
    наш итератор содержит пары язык-предложение, поэтому
    place=0 - это язык-источник, 
    place=1 - это язык-таргет.
    """
    for english, russian in data_iter:
        if place == 0:
            yield token_transform['en'](english)
        else:
            yield token_transform['ru'](russian)

def create_iter(path):
    """
    Создаем итератор по пути файла
    """
    pipe = dp.iter.IterableWrapper([path]) # эта штука нужна, чтобы не загружать содержимое файла целиком
    pipe = dp.iter.FileOpener(pipe, mode='rb') # приготавливаемся к чтению
    pipe = pipe.parse_csv(skip_lines=0, delimiter='\t', as_tuple=True) # парсим файл как тсв (у нас там действительно тсв-формат)
    pipe = pipe.map(lambda row: row[:2]) # оставляем только первые две ячейки, а то там дальше лицензия еще
    return pipe

# Создаем итераторы трейна и валидации. Поскольку у итератора нет длины, а нам она может понадобиться, зададим ее вручную
# (я захардкодила, это, конечно, так себе стратегия)
train = create_iter('train.txt').set_length(383378)
test = create_iter('test.txt').set_length(95845)

Проверим, что все ок:

In [ ]:
next(iter(train))

Собственно словарь собирается с помощью функции build_vocab_from_iterator:

In [56]:
vocab_transform['en'] = build_vocab_from_iterator(
    getTokens(train, 0), # вызываем нашу функцию токенизации
    min_freq=2, # отсеиваем слишком редкие слова
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'], # учитываем, что у нас 4 спецсимвола
    special_first=True) # и что они идут в начале словаря
vocab_transform['ru'] = build_vocab_from_iterator(
    getTokens(train, 1), 
    min_freq=2, 
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'], 
    special_first=True)
# дальше нужно еще установить дефолтный индекс для OOV-слов
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

Также нам понадобится маскировать наши предложения: трансформер не должен видеть правый контекст, он же генерирует новый текст, к тому же, маскировать нужно и паддинги, чтобы они не учитывались при оценке:

<img src="https://cdn.hashnode.com/res/hashnode/image/upload/v1638824585791/vkXCmdGyw.png?auto=compress,format&format=webp" width=500>

In [57]:
def generate_square_subsequent_mask(sz):
    """
    Генерирует маску: она выглядит как матричка-треугольник, где один угол заполнен бесконечностями. 
    """
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    """ 
    Эта функция использует ту, что мы задефайнили выше
    """
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    # заполняем верхний уголок
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len) 
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    # учитываем паддинги
    src_padding_mask = (src == PAD_IDX).transpose(0, 1) 
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Не забудем, что нам необходимо еще позаботиться о функции collate_fn. 

In [58]:
# просто вспомогательная функция-декоратор, чтобы применять другие функции сразу на много элементов
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# добавляет BOS/EOS и создает вектор с индексами инпутов
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# трансформации для ``src`` и ``tgt``, которые собственно превращают наши сырые строки в тензора с индексами
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #токенизация
                                               vocab_transform[ln], #индексация токенов
                                               tensor_transform) # добавили BOS/EOS и создали тензор


# наконец collate_fn
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch: # scr_sample & tgt_sample - это две строки
        # отстрипим \n и прогоним через выше написанные функции
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    # отпадим
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Подготовим себе даталоудеры. 

In [88]:
BATCH_SIZE = 256

train_iter = DataLoader(train, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_iter = DataLoader(test, batch_size=BATCH_SIZE, collate_fn=collate_fn)

#### Seq2seq модель

Пора писать простой seq2seq. Разобьем модель на несколько модулей - Encoder, Decoder и их объединение. 

Encoder должен быть подобен символьной сеточке в POS tagging'е: эмбеддить токены и запускать rnn'ку (в данном случае будем пользоваться GRU) и отдавать последнее скрытое состояние.

Decoder почти такой же, только еще и предсказывает токены на каждом своем шаге.

In [21]:
batch = next(iter(train_iter)) # это нам для тестирования

In [10]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()
        
        self._num_layers = num_layers
        self._hidden_dim = rnn_hidden_dim
        
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=self._hidden_dim, 
                           num_layers=num_layers, dropout=0.2)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        # seq_len, batch_size, 1
        _, h = self._rnn(embs, hidden) # у GRU нет h_c, а output нам не нужен
        return h[-1].unsqueeze(0) # нам нужно только h последнего слоя

In [11]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        outputs, hidden = self._rnn(embs, hidden)
        return self._out(outputs), hidden

Модель перевода будет просто сперва вызывать Encoder, а потом передавать его скрытое состояние декодеру в качестве начального.

In [17]:
class TranslationModel(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, emb_dim=128, 
                 rnn_hidden_dim=256, encoder_layers=2, decoder_layers=1):
        
        super().__init__()
        
        self.encoder = Encoder(source_vocab_size, emb_dim, rnn_hidden_dim, num_layers=encoder_layers)
        self.decoder = Decoder(target_vocab_size, emb_dim, rnn_hidden_dim, num_layers=decoder_layers)
        
    def forward(self, source_inputs, target_inputs):
        encoder_hidden = self.encoder(source_inputs)
        
        return self.decoder(target_inputs, encoder_hidden)

Потестим, работает ли наша архитектурка. batch[0] - это английский текст, batch[1] - русский перевод.

In [22]:
model = TranslationModel(source_vocab_size=len(vocab_transform[SRC_LANGUAGE]), target_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
                        encoder_layers=2).to(DEVICE)

outs = model(batch[0].to(DEVICE), batch[1].to(DEVICE))
outs[0].shape, outs[1].shape

(torch.Size([6, 128, 31707]), torch.Size([1, 128, 256]))

Реализуем простой перевод - жадный. На каждом шаге будем выдавать самый вероятный из предсказываемых токенов:

![](https://github.com/tensorflow/nmt/raw/master/nmt/g3doc/img/greedy_dec.jpg)  
*From [tensorflow/nmt](https://github.com/tensorflow/nmt)*

In [60]:
def greedy_decode(model, source_text):
    model.eval()
    with torch.no_grad():
        result = [] # список индексов предсказываемых токенов
        inputs = text_transform[SRC_LANGUAGE](source_text).to(DEVICE) # затрансформим инпут
        encoder_output = model.encoder(inputs)
        current_input = torch.LongTensor([[BOS_IDX]]).to(DEVICE) # стартовый символ - начало строки
        current_hidden = encoder_output.unsqueeze(0)
        # будем считать, что у нас не может быть длиннее 50 токенов
        for _ in range(50):
            vocab_logits, current_hidden = model.decoder(current_input, current_hidden) 
            current_input = vocab_logits.argmax(dim=-1)
            # но если сгенерился конец строки, то останавливаем
            if current_input.squeeze().item() == EOS_IDX:
                break 

            result.append(current_input)
            
        return ' '.join(vocab_transform[TGT_LANGUAGE].lookup_tokens(result))

Потестим, работает ли наша функция:

In [61]:
greedy_decode(model, "Do you believe?")

'профессионал зла расстоянии облаков природу допрашивают разбираю кабеля переселился попрощаться заговорить такой утомлять переживаете словарём серебряные потеряй разглядеть нечиста погуляла содержимое условия результатами учёным терпимее поручили высказал длинная возбужденной прямолинеен обдумай смеёмся врагами прорыв изучали щекотал прятаться плане планы помогают поспорил встаёшь всеобщее заподозрили Сними обсчитала воображению врёт замёрзнуть избит'

Нужно как-то оценивать модель.

Обычно для этого используется [BLEU скор](https://en.wikipedia.org/wiki/BLEU) - что-то вроде точности угадывания n-gram из правильного (референсного) перевода.

Интерпретируются оценки BLEU следующим образом:

|BLEU Score | Interpretation|
| --- | --- |
|< 10 | Almost useless|
|10 - 19 | Hard to get the gist|
|20 - 29 | The gist is clear, but has significant grammatical errors|
|30 - 40 | Understandable to good translations|
|40 - 50 | High quality translations|
|50 - 60 | Very high quality, adequate, and fluent translations|
|> 60 | Quality often better than human|

<img src="https://cloud.google.com/translate/automl/docs/images/bleu_score_range.png" width=600>

В nltk есть реализация этой метрики. Напишем функцию для оценки модели:

In [81]:
from nltk.translate.bleu_score import corpus_bleu
import numpy as np

def evaluate_model(model, iterator):
    model.eval()
    refs, hyps = [], []
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            encoder_hidden = model.encoder(batch[0].to(DEVICE))

            hidden = encoder_hidden
            result = [torch.LongTensor([BOS_IDX]).expand(1, batch[1].shape[1])]

            for _ in range(30):
                step, hidden = model.decoder(result[-1].to(DEVICE), hidden)
                step = step.argmax(-1)
                result.append(step.cpu())

            targets = batch[1].data.cpu().numpy().T
            _, eos_indices = np.where(targets == EOS_IDX)

            targets = [target[:eos_ind] for eos_ind, target in zip(eos_indices, targets)]

            refs.extend(targets)

            result = torch.cat(result)
            result = result.data.cpu().numpy().T
            _, eos_indices = np.where(result == EOS_IDX)
            result = [res[:eos_ind] for eos_ind, res in zip(eos_indices, result)]
            hyps.extend(result)
            
    return corpus_bleu([[ref] for ref in refs], hyps) * 100

Можно убедиться, что функция работает (и что BLEU у нас... ну... блё)

In [ ]:
evaluate_model(model, test_iter)

Ну и петлю обучения. 

In [68]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch[0].to(DEVICE), batch[1].to(DEVICE))
                
                target = torch.cat((batch[1][1:], batch[1].new_ones((1, batch[1].shape[1])))).to(DEVICE)
                loss = criterion(logits.view(-1, logits.shape[-1]), target.view(-1))

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                # PPX - Perplexity
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, None, name_prefix + '  Val:')
            print('\nVal BLEU = {:.2f}'.format(evaluate_model(model, val_iter)))

Запустим обучение...

In [ ]:
model = TranslationModel(source_vocab_size=len(vocab_transform[SRC_LANGUAGE]), target_vocab_size=len(vocab_transform[TGT_LANGUAGE]), encoder_layers=2).to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=20, val_iter=test_iter)

In [ ]:
greedy_decode(model, "Do you believe?")

## Реализация attention'а

В общем случае, attention работает так: пусть у нас есть набор скрытых состояний $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - представлений слов из исходного языка, полученных с помощью энкодера. И есть некоторое текущее скрытое состояние $\mathbf{h}_i$ - скажем, представление, используемое для предсказания слова на нужном нам языке.

Тогда с помощью аттеншена мы можем получить взвешенное представление контекста $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - вектор $\mathbf{c}_i$:
$$
\begin{align}\begin{split}
\mathbf{c}_i &= \sum\limits_j a_{ij}\mathbf{s}_j\\
\mathbf{a}_{ij} &= \text{softmax}(f_{att}(\mathbf{h}_i, \mathbf{s}_j))
\end{split}\end{align}
$$

$f_{att}$ - функция, которая говорит, насколько хорошо $\mathbf{h}_i$ и $\mathbf{s}_j$ подходят друг другу.

Самые популярные её варианты:
- Additive attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{v}_a{}^\top \text{tanh}(\mathbf{W}_a\mathbf{h}_i + \mathbf{W}_b\mathbf{s}_j)$$
- Dot attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{s}_j$$
- Multiplicative attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{W}_a \mathbf{s}_j$$

In [105]:
class AdditiveAttention(nn.Module):
    def __init__(self, query_size, key_size, hidden_dim):
        super().__init__()

        # query - decoder state, (1, batch, rnn_hidden_dim)
        # value - all encoder states, (seq_len, batch, rnn_hidden_dim)
        # key_proj - self.key_layer(value), (seq_len, batch, hidden_dim)
        # hidden_dim - size of attention layer
        
        self._query_layer = nn.Linear(query_size, hidden_dim)
        self._key_layer = nn.Linear(key_size, hidden_dim)
        self._energy_layer = nn.Linear(hidden_dim, 1)
        
    def forward(self, query, value, mask=None):
        # получаем Key
        key_proj = self._key_layer(value)
        f_att = self._energy_layer(torch.tanh(self._query_layer(query) + key_proj))
        f_att = F.softmax(f_att, 0)
        scores = f_att * value 
        return scores.sum(0), f_att

Обновим декодер и энкодер

In [106]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()
        
        self._num_layers = num_layers
        self._hidden_dim = rnn_hidden_dim
        
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=self._hidden_dim, 
                           num_layers=num_layers, dropout=0.2)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        # seq_len, batch_size, 1
        o, h = self._rnn(embs, hidden) 
        return o, h[-1].unsqueeze(0) # будем аутпуты тоже брать - они нужны для аттеншна

In [125]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, rnn_hidden_dim=256, attn_dim=128, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim + rnn_hidden_dim, 
                           hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size) 
        self._att = AdditiveAttention(rnn_hidden_dim, rnn_hidden_dim, attn_dim) # добавляем слой с аттеншном
        self._drop = nn.Dropout(p=0.3)

    def forward(self, inputs, encoder_output, encoder_mask, hidden=None):
        embs = self._emb(inputs)
        outputs = []
        attentions = []
        for i in range(embs.shape[0]):
            context, f_att = self._att(query=hidden, value=encoder_output, mask=encoder_mask)
            context = context.unsqueeze(0)
            rnn_input = torch.cat((embs[i:i + 1], context), -1)
            output, hidden = self._rnn(rnn_input, hidden)

            outputs.append(output)
            attentions.append(f_att)

        output = self._drop(torch.cat(outputs))
        attentions = torch.cat(attentions)
        return self._out(output), hidden, attentions

Ну и модельку - чуточку. Теперь у нас есть аттеншн и его мерность

In [126]:
class TranslationModel(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, emb_dim=64, rnn_hidden_dim=128, 
                 attn_dim=128, encoder_num_layers=2):
        
        super().__init__()
        
        self.encoder = Encoder(source_vocab_size, emb_dim, rnn_hidden_dim, encoder_num_layers)
        self.decoder = Decoder(target_vocab_size, emb_dim, rnn_hidden_dim, attn_dim, 1)
        
    def forward(self, source_inputs, target_inputs):
        encoder_mask = source_inputs == 1
        encoder_output, encoder_hidden = self.encoder(source_inputs)
        return self.decoder(target_inputs, encoder_output, encoder_mask, encoder_hidden)

Проверим, что архитектурку не поломали

In [ ]:
model = TranslationModel(source_vocab_size=len(vocab_transform[SRC_LANGUAGE]), target_vocab_size=len(vocab_transform[TGT_LANGUAGE])).to(DEVICE)

model(batch[0].to(DEVICE), batch[1].to(DEVICE))

Понадобится немного переписать функцию оценки и петлю:

In [162]:
def evaluate_model(model, iterator):
    model.eval()
    refs, hyps = [], []
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            encoder_output, encoder_hidden = model.encoder(batch[0].to(DEVICE))
            mask = batch[0] == 1.
            
            hidden = encoder_hidden
            result = [torch.LongTensor([BOS_IDX]).expand(1, batch[0].shape[1])]
            
            for _ in range(30):
                step, hidden, _ = model.decoder(result[-1].to(DEVICE), encoder_output, mask, hidden)
                step = step.argmax(-1)
                result.append(step.cpu())
            
            targets = batch[1].data.cpu().numpy().T
            eos_indices = (targets == EOS_IDX).argmax(-1)
            eos_indices[eos_indices == 0] = targets.shape[1]

            targets = [target[:eos_ind] for eos_ind, target in zip(eos_indices, targets)]
            refs.extend(targets)
            
            result = torch.cat(result)
            result = result.data.cpu().numpy().T
            eos_indices = (result == EOS_IDX).argmax(-1)
            eos_indices[eos_indices == 0] = result.shape[1]

            result = [res[:eos_ind] for eos_ind, res in zip(eos_indices, result)]
            hyps.extend(result)
            
    return corpus_bleu([[ref] for ref in refs], hyps) * 100

def do_epoch(model, criterion, data_iter, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                # выхлоп стал больше на один
                logits, _, _ = model(batch[0].to(DEVICE), batch[1].to(DEVICE))
                
                target = torch.cat((batch[1][1:], batch[1].new_ones((1, batch[1].shape[1])))).to(DEVICE)
                loss = criterion(logits.view(-1, logits.shape[-1]), target.view(-1))

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, None, name_prefix + '  Val:')
            print('\nVal BLEU = {:.2f}'.format(evaluate_model(model, val_iter)))

Потестим, что оценка модели работает

In [ ]:
evaluate_model(model, test_iter)

Ну и запустим обучение. На этот раз нам должно понадобиться меньше эпох, чтобы достичь приличного качества

In [ ]:
model = TranslationModel(source_vocab_size=len(vocab_transform[SRC_LANGUAGE]), target_vocab_size=len(vocab_transform[TGT_LANGUAGE])).to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=9, val_iter=test_iter)

Придется переписать и функцию перевода. Заодно давайте отрисуем красивенький график с аттеншном: для этого пусть наша функция возвращает токены исходного предложения, токены результата и аттеншн

In [193]:
def greedy_decode(model, source_text):  
    model.eval()
    with torch.no_grad():
        result, attentions = [], []
        source = token_transform[SRC_LANGUAGE](source_text)
        inputs = text_transform[SRC_LANGUAGE](source_text).to(DEVICE)
        
        encoder_output, encoder_hidden = model.encoder(inputs)
        encoder_mask = torch.zeros_like(inputs).byte()
        
        hidden = encoder_hidden
        step = torch.LongTensor([BOS_IDX]).to(DEVICE)
        
        for _ in range(50):
            step, hidden, attention = model.decoder(step, encoder_output, encoder_mask, hidden)
            step = step.argmax(-1)
            attentions.append(attention)
          
            if step.squeeze().item() == EOS_IDX:
                break
            
            result.append(step.item())   
        result = vocab_transform[TGT_LANGUAGE].lookup_tokens(result)
        return source, result, torch.cat(attentions, -1).data.cpu().numpy()

In [185]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_heatmap(src, trg, scores):

    fig, ax = plt.subplots()
    heatmap = ax.pcolor(scores, cmap='viridis')

    ax.set_xticklabels(trg, minor=False, rotation=45)
    ax.set_yticklabels(src, minor=False)

    ax.xaxis.tick_top()
    ax.set_xticks(np.arange(scores.shape[1]) + 0.5, minor=False)
    ax.set_yticks(np.arange(scores.shape[0]) + 0.5, minor=False)
    ax.invert_yaxis()

    plt.colorbar(heatmap)
    plt.show()

Готово, вы великолепны (но модель нет)

In [ ]:
source, result, attentions = greedy_decode(model, "I didn't pay.")
result

In [ ]:
plot_heatmap(['<s>'] + source, result + ['</s>'], attentions)